In [2]:
import os
os.chdir('../')

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

root_dir = '/data/archive/sd-v1-4'
#models = ['dpm_solver++', 'uni_pc_bh2', 'dpm_solver_v3', 'rbf_order2', 'rbf_order3']
models = ['rbf_order3']
steps = [5, 6, 8, 10, 12, 15, 20]
scales= [1.5, 3.5, 5.5, 7.5]

for scale in scales:
    ref_dir = os.path.join(root_dir, f'dpm_solver++_steps200_scale{scale}')
    for step in steps:
        for model in models:
            comp_dir = os.path.join(root_dir, f'{model}_steps{step}_scale{scale}')
            if not os.path.exists(comp_dir):
                continue
                
            mse_list = []
            for i in range(2000):
                ref_file = os.path.join(ref_dir, f"{i}.pt")
                comp_file = os.path.join(comp_dir, f"{i}.pt")
                if not os.path.exists(ref_file) or not os.path.exists(comp_file):
                    continue
                ref_data = torch.load(ref_file)
                comp_data = torch.load(comp_file)
                mses = torch.sqrt(torch.mean((ref_data['image'] - comp_data['image']) ** 2, dim=[1, 2, 3]))
                mse_list.append(mses)
            mses = torch.cat(mse_list, dim=0)
            print(comp_dir, f"{torch.mean(mses).item():.4f}", f'N={len(mses)}')
        print("")
            

/data/archive/sd-v1-4/rbf_order3_steps5_scale1.5 0.2202 N=10000

